In [1]:
import warnings
warnings.simplefilter('ignore')

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.metrics import confusion_matrix

In [2]:
df = pd.read_csv('Data/2011.csv', skiprows=1, low_memory=False)
df = df.drop(columns=['id', 'member_id'])

df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,5000.0,5000.0,4975.0,36 months,0.11,162.87,B,B2,NaN,10+ years,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,2500.0,2500.0,2500.0,60 months,0.15,59.83,C,C4,Ryder,< 1 year,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,2400.0,2400.0,2400.0,36 months,0.16,84.33,C,C5,NaN,10+ years,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,10000.0,10000.0,10000.0,36 months,0.13,339.31,C,C1,AIR RESOURCES BOARD,10+ years,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,3000.0,3000.0,3000.0,60 months,0.13,67.79,B,B5,University Medical Group,1 year,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.describe()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,url,dti,delinq_2yrs,inq_last_6mths,...,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
count,42535.000000,42535.000000,42535.000000,42535.000000,42535.000000,4.253100e+04,0.0,42535.000000,42506.000000,42506.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,160.000000,160.000000,160.000000
mean,11089.722581,10821.585753,10139.938785,0.121564,322.623063,6.913656e+04,NaN,13.373043,0.152449,1.081424,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4270.794250,49.889875,1.262500
std,7410.938391,7146.914675,7131.598014,0.037024,208.927216,6.409635e+04,NaN,6.726315,0.512406,1.527455,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3119.543917,15.569143,4.142869
min,500.000000,500.000000,0.000000,0.050000,15.670000,1.896000e+03,NaN,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,193.290000,10.690000,0.000000
25%,5200.000000,5000.000000,4950.000000,0.100000,165.520000,4.000000e+04,NaN,8.200000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1842.750000,40.000000,0.000000
50%,9700.000000,9600.000000,8500.000000,0.120000,277.690000,5.900000e+04,NaN,13.470000,0.000000,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3499.350000,49.970000,0.000000
75%,15000.000000,15000.000000,14000.000000,0.150000,428.180000,8.250000e+04,NaN,18.680000,0.000000,2.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5701.100000,60.652500,0.000000
max,35000.000000,35000.000000,35000.000000,0.250000,1305.190000,6.000000e+06,NaN,29.990000,13.000000,33.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14798.200000,92.740000,24.000000


In [4]:
df['emp_length'].head(7)

0    10+ years
1     < 1 year
2    10+ years
3    10+ years
4       1 year
5      3 years
6      8 years
Name: emp_length, dtype: object

In [5]:
df.loan_status.unique()

array(['Fully Paid', 'Charged Off', nan,
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off'],
      dtype=object)

In [6]:
df_using = df[['loan_status', 'loan_amnt','int_rate','annual_inc', 'dti']]

df_using = df_using.dropna()

df_using= df_using.replace(['Does not meet the credit policy. Status:Charged Off','Does not meet the credit policy. Status:Fully Paid'],['Charged Off', 'Fully Paid'])

In [7]:
 # Assign X (data) and y (target)
X = df_using.drop("loan_status", axis=1)
y = df_using["loan_status"]
print(X.shape, y.shape)

(42531, 4) (42531,)


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [9]:
 from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [10]:
 classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [11]:
 print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.8487992977616151
Testing Data Score: 0.8487726887990219


In [12]:
predictions = classifier.predict(X_test)
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

First 10 Predictions:   ['Fully Paid' 'Fully Paid' 'Fully Paid' 'Fully Paid' 'Fully Paid'
 'Fully Paid' 'Fully Paid' 'Fully Paid' 'Fully Paid' 'Fully Paid']
First 10 Actual labels: ['Charged Off', 'Fully Paid', 'Fully Paid', 'Fully Paid', 'Fully Paid', 'Fully Paid', 'Fully Paid', 'Fully Paid', 'Charged Off', 'Fully Paid']


In [13]:
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True).head()

,Prediction,Actual
0,Fully Paid,Charged Off
1,Fully Paid,Fully Paid
2,Fully Paid,Fully Paid
3,Fully Paid,Fully Paid
4,Fully Paid,Fully Paid


In [14]:
confusion_matrix(y_test, predictions)

array([[   0, 1608],
       [   0, 9025]], dtype=int64)

In [15]:
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
print(f"tn: {tn:3d}   fp: {fp:3d}")
print(f"fn: {fn:3d}   tp: {tp:3d}")

tn:   0   fp: 1608
fn:   0   tp: 9025


In [16]:
df_using['loan_status'].value_counts()

Fully Paid     36100
Charged Off     6431
Name: loan_status, dtype: int64